In [1]:
%pip install PyMuPDF python-docx chromadb sentence-transformers transformers gradio pyyaml


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install PyMuPDF python-docx gradio pyyaml

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install rapidfuzz


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.


In [6]:

# --- 0) Imports & setup ---
import os,yaml
from collections import deque

import nltk
nltk.download('punkt', quiet=True)

import gradio as gr

In [7]:
# %% New cell: Generative model imports
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch


In [8]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install huggingface_hub[hf_xet]

Note: you may need to restart the kernel to use updated packages.


In [10]:
# --- 5) Load generation model ---
gen_model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
generator = AutoModelForSeq2SeqLM.from_pretrained(gen_model_name)

In [11]:
# %% New cell: Setup lightweight generation model
gen_model_name = "t5-small"  # smaller, faster for local inference
tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
generator = AutoModelForSeq2SeqLM.from_pretrained(gen_model_name)
device = "cpu"
generator.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [12]:
# --- 6) Helpers for safety & empathy ---
CRISIS_KEYWORDS = ["suicide","self-harm","harm","kill","emergency","overdose","unconscious","not breathing"]

def crisis_check(s: str) -> bool:
    return any(k in s.lower() for k in CRISIS_KEYWORDS)

def empathetic_wrap(answer: str) -> str:
    return "I’m glad you reached out. Caring for a child with autism can be both rewarding and challenging. " + answer

def medical_caution() -> str:
    return "This is not medical advice. Please consult a qualified clinician for treatment decisions."

def add_resource_suggestions(user_q: str):
    if not os.path.exists(RESOURCES_PATH): return []
    try:
        with open(RESOURCES_PATH, "r", encoding="utf-8") as f:
            data = yaml.safe_load(f) or {}
    except Exception: return []
    suggestions = []
    for topic, items in data.items():
        if any(k.lower() in user_q.lower() for k in topic.split(",")):
            for it in items:
                suggestions.append(f"{it.get('name')} — {it.get('url')}")
    return suggestions[:3]

In [13]:
def generate_response(prompt, max_new_tokens=60):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=128).to(device)
    with torch.no_grad():
        outputs = generator.generate(
            **inputs,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            max_new_tokens=max_new_tokens
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [14]:
# %% 
import json
import random
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

# --- Load intents.json ---
INTENTS_FILE = "intents.json"
with open(INTENTS_FILE, "r", encoding="utf-8") as f:
    intents = json.load(f)

patterns, labels, responses = [], [], {}
for intent in intents["intents"]:
    tag = intent["tag"]
    for p in intent["patterns"]:
        patterns.append(p)
        labels.append(tag)
    responses[tag] = intent["responses"]

# --- Encode labels ---
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

# --- Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(patterns, y, test_size=0.2, random_state=42)

# --- Build model pipeline ---
intent_clf = make_pipeline(
    TfidfVectorizer(ngram_range=(1,2), max_features=5000),
    LogisticRegression(max_iter=1000, class_weight="balanced")
)

# --- Train ---
intent_clf.fit(X_train, y_train)

print("✅ Intent model trained")
print(f"Train accuracy: {intent_clf.score(X_train, y_train):.2f}")
print(f"Test accuracy: {intent_clf.score(X_test, y_test):.2f}")

# Store responses globally
intent_responses = responses


✅ Intent model trained
Train accuracy: 0.99
Test accuracy: 0.46


In [15]:
# %%
from rapidfuzz import fuzz, process
import os, re, random, logging
from datetime import datetime
from collections import deque
import pandas as pd

# --- Setup logging ---
logging.basicConfig(
    filename="chat_debug.log",
    level=logging.DEBUG,
    format="%(asctime)s | %(message)s"
)

# --- Files ---
FAQ = {}
FAQ_FILE = "faq_full.csv"
LOG_FILE = "chatbot_log.txt"
UNKNOWN_FILE = "unknown_queries.txt"
last_modified = None

# --- Load FAQ ---
def load_faq():
    global FAQ, last_modified
    try:
        mod_time = os.path.getmtime(FAQ_FILE)
        if last_modified is None or mod_time != last_modified:
            df = pd.read_csv(FAQ_FILE)
            FAQ = dict(zip(df["keyword"], df["response"]))
            last_modified = mod_time
            print("✅ FAQ reloaded from CSV")
    except Exception as e:
        print(f"⚠️ Error loading FAQ: {e}")

# --- Normalize text ---
def normalize(text):
    text = text.lower().strip()
    text = re.sub(r"[^\w\s]", "", text)  # remove punctuation
    text = re.sub(r"\b(children|childs)\b", "child", text)  # unify forms
    text = re.sub(r"s\b", "", text)  # simple plural stripping
    return text

# --- Logging ---
def log_interaction(user_input, bot_response):
    with open(LOG_FILE, "a", encoding="utf-8") as log:
        log.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}]\n")
        log.write(f"User: {user_input}\n")
        log.write(f"Bot: {bot_response}\n\n")

def log_unknown(user_input):
    with open(UNKNOWN_FILE, "a", encoding="utf-8") as f:
        f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {user_input}\n")

# --- FAQ fallback ---
def faq_fallback(user_q: str, threshold=55):
    load_faq()
    q = normalize(user_q)
    if not FAQ:
        return None, 0

    normalized_faq = {normalize(k): v for k, v in FAQ.items()}
    best_match, score, _ = process.extractOne(q, normalized_faq.keys(), scorer=fuzz.token_set_ratio)

    if score >= threshold:
        answer = normalized_faq[best_match]
        log_interaction(user_q, answer)
        return answer, score

    fallback_msg = "I'm not sure about that yet. I’ll note this down so I can learn for next time."
    log_unknown(user_q)
    log_interaction(user_q, fallback_msg)
    return fallback_msg, score

# --- Memory & Chat logic ---
MEMORY = deque(maxlen=4)

def chat_once(user_q: str, intent_threshold=0.65):
    final_answer = ""
    source = "None"

    # --- Step 0: Intent detection ---
    try:
        probs = intent_clf.predict_proba([user_q])[0]
        best_idx = probs.argmax()
        confidence = probs[best_idx]
        intent_label = label_encoder.inverse_transform([best_idx])[0]

        if confidence >= intent_threshold and intent_label in intent_responses:
            final_answer = random.choice(intent_responses[intent_label])
            MEMORY.append((user_q, final_answer))
            log_interaction(user_q, final_answer)
            logging.debug(f"INTENT MATCH | {intent_label} | Confidence={confidence:.2f} → {final_answer}")
            return final_answer
        else:
            logging.debug(f"INTENT LOW CONF | Best={intent_label} | Confidence={confidence:.2f} < {intent_threshold}")
    except Exception as e:
        logging.debug(f"Intent detection error: {e}")

    # --- Step 1: FAQ check ---
    faq_answer, score = faq_fallback(user_q)
    if faq_answer:
        final_answer = faq_answer
        source = "FAQ" if score >= 55 else "Fallback"

    # --- Step 1b: Generative fallback (always triggers if FAQ/intents fail) ---
    if not final_answer or "I’m not sure" in final_answer or "enough information" in final_answer:
        try:
            gen_answer = generate_response(user_q)
            final_answer = empathetic_wrap(gen_answer)  # optional: add empathy
            source = "Generative"
            logging.debug(f"GENERATIVE FALLBACK | Generated response: {final_answer}")
        except Exception as e:
            logging.debug(f"Generative model error: {e}")
            if not final_answer:
                final_answer = "I'm sorry, I don't have enough information to answer that."

    # --- Step 2: Add warnings for sensitive topics ---
    if any(w in user_q.lower() for w in ["diagnose","medication","therapy","treatment"]):
        final_answer += " (⚠️ Please consult a medical professional for advice.)"

    # --- Step 3: Memory + logging ---
    MEMORY.append((user_q, final_answer))
    log_interaction(user_q, final_answer)
    logging.debug(f"CHAT ROUTE | Source={source} | Query='{user_q}' | Answer='{final_answer[:100]}'")

    return final_answer


In [18]:
# --- 9) Gradio interface ---
with gr.Blocks(title="Rafiki Assist") as demo:
    gr.Markdown("### Rafiki Assist — Caregiver Support Chatbot")
    chat = gr.Chatbot(height=380)
    msg = gr.Textbox(placeholder="Ask about routines, meltdowns, communication, etc.")
    clear = gr.Button("Clear")

    def respond(user_message, history):
        answer = chat_once(user_message)
        history = history + [[user_message, answer]]
        return history, ""

    msg.submit(respond, [msg, chat], [chat, msg])
    clear.click(lambda: ([], ""), None, [chat, msg])

demo.launch()

C:\Users\USER\AppData\Local\Temp\ipykernel_9724\183835957.py:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat = gr.Chatbot(height=380)


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


✅ FAQ reloaded from CSV


In [ ]:
# %%
import json

def retrain_intent_model():
    """
    Reload intents.json and retrain the intent classifier.
    Use this after you add new patterns/intents to intents.json.
    """
    global intent_clf, label_encoder, intent_responses

    try:
        with open(INTENTS_FILE, "r", encoding="utf-8") as f:
            intents = json.load(f)

        patterns, labels, responses = [], [], {}
        for intent in intents["intents"]:
            tag = intent["tag"]
            for p in intent["patterns"]:
                patterns.append(p)
                labels.append(tag)
            responses[tag] = intent["responses"]

        # Encode labels again
        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(labels)

        # Re-train
        intent_clf = make_pipeline(
            TfidfVectorizer(ngram_range=(1,2), max_features=5000),
            LogisticRegression(max_iter=1000, class_weight="balanced")
        )
        intent_clf.fit(patterns, y)

        intent_responses = responses
        print("✅ Intent model retrained with updated intents.json")
        return True

    except Exception as e:
        print(f"⚠️ Retrain error: {e}")
        return False


def show_unknown_queries(limit=10):
    """
    Quickly preview the most recent unknown queries
    so you can decide which to add to intents.json.
    """
    if not os.path.exists(UNKNOWN_FILE):
        print("ℹ️ No unknown queries logged yet.")
        return []
    
    with open(UNKNOWN_FILE, "r", encoding="utf-8") as f:
        lines = f.readlines()
    recent = lines[-limit:]
    for l in recent:
        print(l.strip())
    return recent


✅ FAQ reloaded from CSV
